In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model

In [2]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [3]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [4]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [5]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [6]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [7]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

In [9]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [10]:
trainx_dhw, trainy_dhw = buildDataSet(trainX_dhw, trainY_dhw, 3)
testx_dhw, testy_dhw = buildDataSet(testX_dhw, testY_dhw, 3)
trainx_elec, trainy_elec = buildDataSet(trainX_elec, trainY_elec, 3)
testx_elec, testy_elec = buildDataSet(testX_elec, testY_elec, 3)
trainx_n_elec, trainy_n_elec = buildDataSet(trainX_n_elec, trainY_n_elec, 3)
testx_n_elec, testy_n_elec = buildDataSet(testX_n_elec, testY_n_elec, 3)

## Reverse MinMaxScaler

In [11]:
# reverse_train_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_train_x)
# reverse_train_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_train_y)
# reverse_test_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_test_x)
# reverse_test_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_test_y)

# rev_dhw_train_x = pd.DataFrame(reverse_train_x, index=dhw_x_train.index, columns=dhw_x_train.columns)
# rev_dhw_train_y = pd.DataFrame(reverse_train_y, index=dhw_y_train.index, columns=dhw_y_train.columns)

# rev_dhw_test_x = pd.DataFrame(reverse_test_x, index=dhw_x_test.index, columns=dhw_x_test.columns)
# rev_dhw_test_y = pd.DataFrame(reverse_test_y, index=dhw_y_test.index, columns=dhw_y_test.columns)

# rev_dhw_trainSet = pd.concat([rev_dhw_train_x, rev_dhw_train_y], axis=1)
# rev_dhw_testSet = pd.concat([rev_dhw_test_x, rev_dhw_test_y], axis=1)

In [28]:
model_for_elec = load_model('N_elec_Source.h5')

In [29]:
for layer in model_for_elec.layers:
    print('Name:', layer.name, 'Trainable:', layer.trainable)

Name: input_2 Trainable: True
Name: conv1d_3 Trainable: True
Name: max_pooling1d_3 Trainable: True
Name: conv1d_4 Trainable: True
Name: max_pooling1d_4 Trainable: True
Name: conv1d_5 Trainable: True
Name: max_pooling1d_5 Trainable: True
Name: lstm_6 Trainable: True
Name: lstm_7 Trainable: True
Name: lstm_8 Trainable: True
Name: lstm_9 Trainable: True
Name: lstm_10 Trainable: True
Name: lstm_11 Trainable: True
Name: flatten_1 Trainable: True
Name: dense_1 Trainable: True


In [30]:
for layer in model_for_elec.layers[:7]:
    layer.trainable = False

for layer in model_for_elec.layers:
    print('Name:', layer.name, 'Trainable:', layer.trainable)

Name: input_2 Trainable: False
Name: conv1d_3 Trainable: False
Name: max_pooling1d_3 Trainable: False
Name: conv1d_4 Trainable: False
Name: max_pooling1d_4 Trainable: False
Name: conv1d_5 Trainable: False
Name: max_pooling1d_5 Trainable: False
Name: lstm_6 Trainable: True
Name: lstm_7 Trainable: True
Name: lstm_8 Trainable: True
Name: lstm_9 Trainable: True
Name: lstm_10 Trainable: True
Name: lstm_11 Trainable: True
Name: flatten_1 Trainable: True
Name: dense_1 Trainable: True


In [31]:
# 모델 컴파일
earlystopping = EarlyStopping(monitor='loss', patience=50)
checkpoint = ModelCheckpoint(moniter='loss',filepath='TL_nelec_to_elec.h5')
model_for_elec.compile(loss='mae', optimizer=tf.keras.optimizers.Adamax(learning_rate=3e-4,clipnorm=1.0), metrics=['mae'])
# 모델 요약 정보 출력
hist = model_for_elec.fit(trainx_elec, trainy_elec, epochs=1000, batch_size=1024, callbacks=[earlystopping,checkpoint])

Epoch 1/1000


3/3 [==============================] - 3s 58ms/step - loss: 2.8094 - mae: 2.8094
Epoch 2/1000
3/3 [==============================] - 0s 38ms/step - loss: 1.4755 - mae: 1.4755
Epoch 3/1000
3/3 [==============================] - 0s 43ms/step - loss: 0.5666 - mae: 0.5666
Epoch 4/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.3649 - mae: 0.3649
Epoch 5/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.4172 - mae: 0.4172
Epoch 6/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.4212 - mae: 0.4212
Epoch 7/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.3802 - mae: 0.3802
Epoch 8/1000
3/3 [==============================] - 0s 41ms/step - loss: 0.3388 - mae: 0.3388
Epoch 9/1000
3/3 [==============================] - 0s 41ms/step - loss: 0.3325 - mae: 0.3325
Epoch 10/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.3396 - mae: 0.3396
Epoch 11/1000
3/3 [==============================] - 0s 35ms/step - loss

In [32]:
model = load_model('TL_nelec_to_elec.h5')
pred = model.predict(testx_elec) 
pred = np.reshape(pred,(-1,1))

real = np.array(testy_elec)

40/40 [==============================] - 1s 6ms/step


In [33]:
from sklearn.metrics import mean_absolute_error
print('MAE : ', round(mean_absolute_error(real, pred),4)) 

from sklearn.metrics import mean_squared_error
print('MSE : ', round(mean_squared_error(real, pred),4))

from sklearn.metrics import r2_score 
print('R2 : ', round(r2_score(real, pred),4))

MAE :  0.1413
MSE :  0.0339
R2 :  0.3958
